In [3]:
# --------------------------------------------
# Step 1: Import required libraries
# --------------------------------------------
import pandas as pd
import numpy as np
import seaborn as sns

# Display settings for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [4]:
# --------------------------------------------
# Step 2: Load dataset
# --------------------------------------------
df = sns.load_dataset('tips')

df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [6]:
# --------------------------------------------
# Step 3: Basic Dataset Overview
# --------------------------------------------

print("Shape of dataset:", df.shape)
print("\nDataset Info:\n")
df.info()

print("\nMissing Values:\n")
df.isnull().sum()

Shape of dataset: (244, 7)

Dataset Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB

Missing Values:



,0
total_bill,0
tip,0
sex,0
smoker,0
day,0
time,0
size,0


In [14]:
# --------------------------------------------
# Step 4: Basic GroupBy Operations
# --------------------------------------------

# Average tip amount by gender
avg_tip_by_gender = df.groupby('sex')['tip'].mean()
avg_tip_by_gender

/tmp/ipython-input-1760575026.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_tip_by_gender = df.groupby('sex')['tip'].mean()


,tip
sex,
Male,3.09
Female,2.83


In [15]:
# Average tip amount by smoker status
avg_tip_by_smoker = df.groupby('smoker')['tip'].mean()
avg_tip_by_smoker

/tmp/ipython-input-1215998593.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_tip_by_smoker = df.groupby('smoker')['tip'].mean()


,tip
smoker,
Yes,3.01
No,2.99


In [16]:
# --------------------------------------------
# Step 5: Multiple Aggregations using agg()
# --------------------------------------------

# Aggregating multiple statistics at once
bill_stats = df.groupby('day').agg(
    avg_bill=('total_bill', 'mean'),
    max_bill=('total_bill', 'max'),
    min_bill=('total_bill', 'min'),
    total_tips=('tip', 'sum')
)

bill_stats

/tmp/ipython-input-819284331.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bill_stats = df.groupby('day').agg(


,avg_bill,max_bill,min_bill,total_tips
day,,,,
Thur,17.68,43.11,7.51,171.83
Fri,17.15,40.17,5.75,51.96
Sat,20.44,50.81,3.07,260.40
Sun,21.41,48.17,7.25,247.39


In [17]:
# --------------------------------------------
# Step 6: Grouping by Multiple Columns
# --------------------------------------------

# Average total bill grouped by day and time
day_time_bill = df.groupby(['day', 'time'])['total_bill'].mean()
day_time_bill

/tmp/ipython-input-73068192.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  day_time_bill = df.groupby(['day', 'time'])['total_bill'].mean()


day   time  
Thur  Lunch    17.66
      Dinner   18.78
Fri   Lunch    12.85
      Dinner   19.66
Sat   Lunch      NaN
      Dinner   20.44
Sun   Lunch      NaN
      Dinner   21.41
Name: total_bill, dtype: float64

In [18]:
# --------------------------------------------
# Step 7: Resetting Index for Clean Output
# --------------------------------------------

day_time_bill_df = day_time_bill.reset_index()
day_time_bill_df

,day,time,total_bill
0,Thur,Lunch,17.66
1,Thur,Dinner,18.78
2,Fri,Lunch,12.85
3,Fri,Dinner,19.66
4,Sat,Lunch,NaN
5,Sat,Dinner,20.44
6,Sun,Lunch,NaN
7,Sun,Dinner,21.41


In [19]:
# --------------------------------------------
# Step 8: Using transform() for Row-Level Metrics
# --------------------------------------------

# Add average tip per dayto each row
df['avg_tip_per_day'] = df.groupby('day')['tip'].transform('mean')

df[['day', 'tip', 'avg_tip_per_day']].head()

/tmp/ipython-input-976012969.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['avg_tip_per_day'] = df.groupby('day')['tip'].transform('mean')


,day,tip,avg_tip_per_day
0,Sun,1.01,3.26
1,Sun,1.66,3.26
2,Sun,3.50,3.26
3,Sun,3.31,3.26
4,Sun,3.61,3.26


In [20]:
# --------------------------------------------
# Step 9: Creating Business Metrics
# --------------------------------------------

# Tip percentage
df['tip_percentage'] = (df['tip'] / df['total_bill']) * 100

df[['total_bill', 'tip', 'tip_percentage']].head()

,total_bill,tip,tip_percentage
0,16.99,1.01,5.94
1,10.34,1.66,16.05
2,21.01,3.50,16.66
3,23.68,3.31,13.98
4,24.59,3.61,14.68


In [21]:
# --------------------------------------------
# Step 10: Pivot Table Analysis
# --------------------------------------------

# Average tip percentage by day and time
pivot_tip = pd.pivot_table(
    df,
    values='tip_percentage',
    index='day',
    columns='time',
    aggfunc='mean'
)

pivot_tip

/tmp/ipython-input-2110740223.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_tip = pd.pivot_table(


time,Lunch,Dinner
day,,
Thur,16.13,15.97
Fri,18.88,15.89
Sat,NaN,15.32
Sun,NaN,16.69


In [23]:
# --------------------------------------------
# Step 11: Crosstab for Categorical Analysis
# --------------------------------------------

# Count of smokers vs non- smokers by day
smoker_dy_ct = pd.crosstab(df['day'], df['smoker'])
smoker_dy_ct

smoker,Yes,No
day,,
Thur,17,45
Fri,15,4
Sat,42,45
Sun,19,57


In [25]:
# --------------------------------------------
# Step 12: Ranking within Groups
# --------------------------------------------

# Rank total bills within each day
df['bill_rank_per_day']  = df.groupby('day')['total_bill'].rank(ascending=False)

df[['day', 'total_bill', 'bill_rank_per_day']].head()

/tmp/ipython-input-1702185816.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['bill_rank_per_day']  = df.groupby('day')['total_bill'].rank(ascending=False)


,day,total_bill,bill_rank_per_day
0,Sun,16.99,49.00
1,Sun,10.34,68.00
2,Sun,21.01,34.00
3,Sun,23.68,27.00
4,Sun,24.59,23.00


In [26]:
# --------------------------------------------
# Step 13: Custom Aggregation using lambda
# --------------------------------------------

# Custom metric: range of total bill per day
custom_agg = df.groupby('day').agg(
    bill_range=('total_bill', lambda x: x.max() - x.min())
)

custom_agg

/tmp/ipython-input-3329863091.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  custom_agg = df.groupby('day').agg(


,bill_range
day,
Thur,35.60
Fri,34.42
Sat,47.74
Sun,40.92


In [27]:
# --------------------------------------------
# Step 14: Final Insights Summary
# --------------------------------------------

summary = df.groupby('day').agg(
    avg_bill=('total_bill', 'mean'),
    avg_tip=('tip', 'mean'),
    avg_tip_pct=('tip_percentage', 'mean'),
    customers=('total_bill', 'count')
)

summary

/tmp/ipython-input-3754182566.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = df.groupby('day').agg(


,avg_bill,avg_tip,avg_tip_pct,customers
day,,,,
Thur,17.68,2.77,16.13,62
Fri,17.15,2.73,16.99,19
Sat,20.44,2.99,15.32,87
Sun,21.41,3.26,16.69,76
